<a href="https://colab.research.google.com/github/v-grand/Collector/blob/main/Crypto_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal

In [1]:
!pip install ta
!pip install mplfinance

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2c24619b3fa125c627d045fb47e8a01728de26097686f7353b36ec8694016085
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.9 MB/s eta 0:00:00


### Import

In [2]:

'''
%pip install prophet

'''
import requests
import urllib

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

import pandas as pd
from prophet import Prophet

from datetime import datetime, timedelta
import numpy as np
from pandas import DataFrame
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
import ta
from typing import Tuple
import yfinance as yf

from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error


from google.colab import drive
#from tensorflow.keras.models import load_model

from prophet import Prophet

# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Function

In [3]:
def prep_data(drop_na:bool = True, ticker='BTC-USD', period='60d', interval_='5m') -> DataFrame:

    N_pred=3

    data = yf.download(ticker, period=period, interval=interval_)


    data.drop(columns=['Adj Close','Volume'], inplace=True)

    data['NextOpen'] = data['Open'].shift(-1)
    data['NextHigh'] = data['High'].shift(-1)
    data['NextLow'] = data['Low'].shift(-1)
    data['NextClose'] = data['Close'].shift(-1)


    for i in range(1, N_pred):
      data[f'PrevOpen_bar{i}'] = data['Open'].shift(i)
      data[f'PrevHigh_bar{i}'] = data['High'].shift(i)
      data[f'PrevLow_bar{i}'] = data['Low'].shift(i)
      data[f'PrevClose_bar{i}'] = data['Close'].shift(i)

    data = data.iloc[N_pred:]


    # Drop NaN values
    if(drop_na):
        data.dropna(inplace=True)

    data.reset_index(inplace=True)

    if data.empty:
      print(f'No data found for {ticker}')
      return None
    else:

      if 'Date' in data.columns:
        last_date_value= data['Date'].iloc[-1]
        first_data_value=data['Date'].iloc[0]
        print(f'   {ticker} load at {first_data_value} to {last_date_value} interval {interval_}')

      elif 'Datetime' in data.columns:
        last_date_value= data['Datetime'].iloc[-1]
        first_data_value=data['Datetime'].iloc[0]
        print(f'   {ticker} load at {first_data_value} to {last_date_value} interval {interval_}')

    data.rename(columns=lambda x: f'{x}_{interval_}', inplace=True)

    return data


In [4]:
def scale_data(data:DataFrame, columns_x, columns_y) -> Tuple[MinMaxScaler, np.ndarray, np.ndarray]:

    # Prepare target variable
    y = data[columns_y].values
    X = data [columns_x]

    # Scale features
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).reshape(-1)

    return scaler, X_scaled, y_scaled

# EXTRACT

In [19]:
#[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
interval_ = '3mo'
period_ = 'max'  # или другой поддерживаемый период, например '10y' для последних 10 лет

ticker_ = 'BTC-USD'


In [20]:
data = yf.download(tickers=ticker_, period=period_, interval=interval_)

[*********************100%***********************]  1 of 1 completed


In [28]:
import yfinance as yf
import pandas as pd
import os

def extract_yf_data(ticker, periods, intervals):
    """
    Загружает данные для всех указанных интервалов и возвращает их в виде словаря.

    :param ticker: Символ тикера, например, 'BTC-USD'.
    :param periods: Период загрузки данных, например, '10y', 'max'.
    :param intervals: Список интервалов для загрузки данных, например, ['1m', '5m', '1d'].
    :return: Словарь с данными для каждого интервала.
    """
    datasets = {}

    for interval in intervals:
        try:
            print(f"Загрузка данных для интервала {interval}...")
            data = yf.download(tickers=ticker, period=periods, interval=interval)
            if data.empty:
                print(f"Нет данных для интервала {interval}. Пропускаем.")
                continue

            # Добавляем столбец с интервалом для удобства
            data['interval'] = interval

            # Сохраняем данные в словарь
            datasets[interval] = data
            print(interval)
            print(datasets[interval].head())

        except Exception as e:
            print(f"Ошибка при загрузке данных для интервала {interval}: {e}")

    print("Загрузка завершена.")
    return datasets

# Пример использования
ticker_ = 'BTC-USD'
period_ = 'max'
intervals_ = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']

# Загрузка данных
datasets = (ticker_, period_, intervals_)


Загрузка данных для интервала 1m...


[*********************100%***********************]  1 of 1 completed


1m
Price                              Close           High            Low  \
Ticker                           BTC-USD        BTC-USD        BTC-USD   
Datetime                                                                 
2025-01-17 09:50:00+00:00  101808.421875  101808.421875  101808.421875   
2025-01-17 09:51:00+00:00  101745.015625  101745.015625  101745.015625   
2025-01-17 09:53:00+00:00  101818.250000  101818.250000  101818.250000   
2025-01-17 09:54:00+00:00  101845.039062  101845.039062  101845.039062   
2025-01-17 09:55:00+00:00  101908.031250  101908.031250  101908.031250   

Price                               Open   Volume interval  
Ticker                           BTC-USD  BTC-USD           
Datetime                                                    
2025-01-17 09:50:00+00:00  101808.421875        0       1m  
2025-01-17 09:51:00+00:00  101745.015625        0       1m  
2025-01-17 09:53:00+00:00  101818.250000  4771840       1m  
2025-01-17 09:54:00+00:00  101845.0390

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BTC-USD']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (2m 1926-02-18 09:49:27+00:00 -> 2025-01-24 09:49:27+00:00) (Yahoo error = "2m data not available for startTime=1410912000 and endTime=1737712167. The requested range must be within the last 60 days.")')


Нет данных для интервала 2m. Пропускаем.
Загрузка данных для интервала 5m...


...

60m
Price                       Close          High           Low          Open  \
Ticker                    BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Datetime                                                                      
2023-01-25 09:00:00  22641.953125  22690.357422  22615.089844  22682.052734   
2023-01-25 10:00:00  22624.876953  22640.937500  22551.417969  22640.937500   
2023-01-25 11:00:00  22599.845703  22630.679688  22586.701172  22625.150391   
2023-01-25 12:00:00  22695.574219  22736.791016  22600.078125  22600.078125   
2023-01-25 13:00:00  22612.806641  22727.470703  22612.806641  22695.933594   

Price                   Volume interval  
Ticker                 BTC-USD           
Datetime                                 
2023-01-25 09:00:00          0      60m  
2023-01-25 10:00:00  209887232      60m  
2023-01-25 11:00:00          0      60m  
2023-01-25 12:00:00  310562816      60m  
2023-01-25 13:00:00          0      60m  
Загрузка данных для интерва

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1h
Price                             Close          High           Low  \
Ticker                          BTC-USD       BTC-USD       BTC-USD   
Datetime                                                              
2023-01-25 09:00:00+00:00  22641.953125  22690.357422  22615.089844   
2023-01-25 10:00:00+00:00  22624.876953  22640.937500  22551.417969   
2023-01-25 11:00:00+00:00  22599.845703  22630.679688  22586.701172   
2023-01-25 12:00:00+00:00  22695.574219  22736.791016  22600.078125   
2023-01-25 13:00:00+00:00  22612.806641  22727.470703  22612.806641   

Price                              Open     Volume interval  
Ticker                          BTC-USD    BTC-USD           
Datetime                                                     
2023-01-25 09:00:00+00:00  22682.052734          0       1h  
2023-01-25 10:00:00+00:00  22640.937500  209887232       1h  
2023-01-25 11:00:00+00:00  22625.150391          0       1h  
2023-01-25 12:00:00+00:00  22600.078125  310562816      

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


5d
Price            Close        High         Low        Open    Volume interval
Ticker         BTC-USD     BTC-USD     BTC-USD     BTC-USD   BTC-USD         
Date                                                                         
2014-09-17  457.334015  468.174011  452.421997  465.864014  21056800       5d
2014-09-22  402.152008  406.915985  397.130005  399.100006  24127600       5d
2014-09-27  399.519989  406.622986  397.372009  403.556000  15029300       5d
2014-10-02  375.071991  385.497009  372.946014  383.988007  21777700       5d
2014-10-07  336.187012  339.247009  320.481995  330.584015  49199900       5d
Загрузка данных для интервала 1wk...
1wk
Price            Close        High         Low        Open     Volume interval
Ticker         BTC-USD     BTC-USD     BTC-USD     BTC-USD    BTC-USD         
Date                                                                          
2014-09-15  398.821014  468.174011  384.532013  465.864014  156903400      1wk
2014-09-22  377.

In [30]:
datasets['1m'].head()

Price,Close,High,Low,Open,Volume,interval
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,
Datetime,,,,,,
2025-01-17 09:50:00+00:00,101808.421875,101808.421875,101808.421875,101808.421875,0,1m
2025-01-17 09:51:00+00:00,101745.015625,101745.015625,101745.015625,101745.015625,0,1m
2025-01-17 09:53:00+00:00,101818.250000,101818.250000,101818.250000,101818.250000,4771840,1m
2025-01-17 09:54:00+00:00,101845.039062,101845.039062,101845.039062,101845.039062,0,1m
2025-01-17 09:55:00+00:00,101908.031250,101908.031250,101908.031250,101908.031250,6283264,1m


In [ ]:
columns_x = ['Open_3mo', 'High_3mo', 'Low_3mo', 'Close_3mo',
           'PrevOpen_bar1_3mo', 'PrevHigh_bar1_3mo', 'PrevLow_bar1_3mo',
           'PrevClose_bar1_3mo', 'PrevOpen_bar2_3mo', 'PrevHigh_bar2_3mo',
           'PrevLow_bar2_3mo', 'PrevClose_bar2_3mo']
columns_y = ['NextClose_3mo']
# ['NextOpen_3mo', 'NextHigh_3mo', 'NextLow_3mo', 'NextClose_3mo']
scaler, X_scaled, y_scaled = scale_data(data_3mo, columns_x, columns_y)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

model_name = "/content/drive/My Drive/High_random_forest.joblib"
    # Random Forest model
model_High = RandomForestRegressor(n_estimators=100, random_state=42)
model_High.fit(X_train, y_train)

    # Save the model to Google Drive
joblib.dump(model_High, model_name)

    # Predict the prices
predicted_prices = model_High.predict(X_test)

    # Evaluate
mse = mean_squared_error(y_test, predicted_prices)
print(f'Mean Squared Error: {mse}')
#feature_importance_high = model_High.feature_importances_
#feature_df_high = pd.DataFrame({'Feature': data_x.columns, 'Importance': feature_importance_high})

In [ ]:
# developer: Foundry Digital
# Copyright © 2023 Foundry Digital

# Import necessary modules to use for model creation - can be downloaded using pip
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def create_and_save_base_model_lstm(scaler:MinMaxScaler, X_scaled:np.ndarray, y_scaled:np.ndarray) -> float:
    """
    Base model that can be created for predicting the S&P 500 close price

    The function creates a base model, given a scaler, inputs and outputs, and
    stores the model weights as a .h5 file in the mining_models/ folder. The model
    architecture and model name given now is a placeholder, can (and should)
    be changed by miners to build more robust models.

    Input:
        :param scaler: The scaler used to scale the inputs during model training process
        :type scaler: sklearn.preprocessing.MinMaxScaler

        :param X_scaled: The already scaled input data that will be used by the model to train and test
        :type X_scaled: np.ndarray

        :param y_scaled: The already scaled output data that will be used by the model to train and test
        :type y_scaled: np.ndarray

    Output:
        :returns: The MSE of the model on the test data
        :rtype: float
    """
    model_name = "mining_models/base_lstm"

    # Reshape input for LSTM
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # LSTM model - all hyperparameters are baseline params - should be changed according to your required
    # architecture. LSTMs are also not the only way to do this, can be done using any algo deemed fit by
    # the creators of the miner.
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_scaled.shape[1], X_scaled.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32)
    model.save(f'{model_name}.h5')

    # Predict the prices - this is just for a local test, this prediction just allows
    # miners to assess the performance of their models on real data.
    predicted_prices = model.predict(X_test)

    # Rescale back to original range
    predicted_prices = scaler.inverse_transform(predicted_prices)
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate
    mse = mean_squared_error(y_test_rescaled, predicted_prices)
    print(f'Mean Squared Error: {mse}')

    return mse

def create_and_save_base_model_regression(scaler:MinMaxScaler, X_scaled:np.ndarray, y_scaled:np.ndarray) -> float:
    """
    Base model that can be created for predicting the S&P 500 close price

    The function creates a base model, given a scaler, inputs and outputs, and
    stores the model weights as a .h5 file in the mining_models/ folder. The model
    architecture and model name given now is a placeholder, can (and should)
    be changed by miners to build more robust models.

    Input:
        :param scaler: The scaler used to scale the inputs during model training process
        :type scaler: sklearn.preprocessing.MinMaxScaler

        :param X_scaled: The already scaled input data that will be used by the model to train and test
        :type X_scaled: np.ndarray

        :param y_scaled: The already scaled output data that will be used by the model to train and test
        :type y_scaled: np.ndarray

    Output:
        :returns: The MSE of the model on the test data
        :rtype: float
    """
    model_name = "mining_models/base_linear_regression"

    # Split data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # LSTM model - all hyperparameters are baseline params - should be changed according to your required
    # architecture. LSTMs are also not the only way to do this, can be done using any algo deemed fit by
    # the creators of the miner.
    model = LinearRegression()
    model.fit(X_train, y_train)

    '''with h5py.File(f'{model_name}.h5', 'w') as hf:
        hf.create_dataset('coefficients', data=model.coef_)
        hf.create_dataset('intercept', data=model.intercept_)'''
    joblib.dump(model, f"{model_name}.joblib")

    # Predict the prices - this is just for a local test, this prediction just allows
    # miners to assess the performance of their models on real data.
    predicted_prices = model.predict(X_test)

    # Rescale back to original range
    predicted_prices = scaler.inverse_transform(predicted_prices.reshape(-1, 1))
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Evaluate
    mse = mean_squared_error(y_test_rescaled, predicted_prices)
    print(f'Mean Squared Error: {mse}')

    return mse

# Spread

In [ ]:
interval_='1d'
period_='300d'

ticker_2='XRP-USD'
ticker_1='ATOM-USD'
K_1=1
K_2=20

data_1 = prep_data(drop_na=True, ticker=ticker_1, period=period_, interval_=interval_)
#print(data_1.shape)

data_2 = prep_data(drop_na=True, ticker=ticker_2, period=period_, interval_=interval_)
#print(data_2.shape)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

   ATOM-USD load at 2019-03-17 00:00:00 to 2025-01-02 00:00:00 interval 1d
   XRP-USD load at 2017-11-12 00:00:00 to 2025-01-02 00:00:00 interval 1d


In [ ]:
# Объединяем датасеты по столбцу Datetime
combined_df = pd.merge(data_1, data_2, on='Datetime_1h', how='inner')

combined_df['Close']=K_1*combined_df['Close_1h_x']-K_2*combined_df['Close_1h_y']
#combined_df.info()

TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

In [ ]:
from prophet import Prophet

dp_ds = combined_df[['Datetime_1h', 'Close']].copy()
dp_ds['Datetime_1h'] = dp_ds['Datetime_1h'].dt.tz_localize(None)

# Подготовка данных для Prophet
dp_ds = dp_ds.rename(columns={'Datetime_1h': 'ds', 'Close': 'y'})


model = Prophet(
    growth='linear',
    changepoint_range=0.8,
    seasonality_mode='additive',
    seasonality_prior_scale=10.0,
    holidays_prior_scale=10.0,
    changepoint_prior_scale=0.05,
    interval_width=0.8,
    uncertainty_samples=10
)
# Продолжаем обучение и прогнозирование

model.fit(dp_ds)

# Создание будущего DataFrame для прогнозов
future = model.make_future_dataframe(periods=30)  # Прогноз на 30 дней вперед

# Получение прогнозов
forecast = model.predict(future)



In [ ]:
# Построение графика прогнозов
fig = model.plot( forecast.iloc[-100:])
plt.xlabel(f'Date')
plt.ylabel('Quote')
plt.title(f'{ticker_1}x{K_1} and {ticker_2}x{K_2}')
plt.show()